In [2]:
import pandas as pd 
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
#from geopy import distance

In [130]:
df = pd.read_csv('Baseball_Merged.csv')


In [177]:
# RateLimiter must be used for Nominatim
geocoder = RateLimiter(Nominatim(user_agent='ba').geocode, min_delay_seconds=1)
venues = df['venue_name'].unique()

location_dict = {}

for i in venues: 
    location_dict[i] = geocoder(i)


In [178]:
# there are erroneous pairings
error_pairs = {}
for venue in venues:
    try:
        if location_dict[venue].raw['type'] != 'stadium':
            error_pairs[venue] = location_dict[venue].raw['type']
    except:
        error_pairs[venue] = 'None'

# When looking at online, found good string for geopy
rename_dict = {'Miller Park':'American Family Field',
'O.co Coliseum': 'RingCentral Coliseum',
'U.S. Cellular Field' : 'Guaranteed Rate Field',
'AT&T Park' : 'Oracle Park',
'Oakland Coliseum' : 'RingCentral Coliseum',
'Fort Bragg Field' : 'Fort Bragg Field baseball',
'Williamsport Little League Classic' : 'Williamsport, Pennsylvania',
'Angel Stadium': 'Angel Stadium of Anaheim',
'BB&T Ballpark': 'Williamsport, Pennsylvania'
}

In [185]:
# Get locations to each venue
for k,v in rename_dict.items():
    location_dict[v] = geocoder(v)

for key in [k for k,v in rename_dict.items()]:
    if key in location_dict:
        del location_dict[key]

In [187]:
# we need all date-venue_name pairs for the API 
subdf = df[['date','venue_name']].drop_duplicates()

# we require a new col where new venue names are used
to_replace = [k for k,v in rename_dict.items()]

new_venue = []

for i in subdf['venue_name']:
    if i in to_replace:
        new_venue.append(rename_dict[i])
    else:
        new_venue.append(i)

subdf['new_venue'] = new_venue

# map location based on new venue
subdf['location'] = [location_dict[i] for i in subdf['new_venue']]

In [199]:
sth = geocoder('Wrigley Field')

In [207]:
sth.raw

{'place_id': 258157221,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 1407988,
 'boundingbox': ['41.9473101', '41.9490224', '-87.6567241', '-87.6544038'],
 'lat': '41.9481225',
 'lon': '-87.6563513145702',
 'display_name': 'Wrigley Field, 1060, West Addison Street, Wrigleyville, Lake View, Chicago, Lake View Township, Cook County, Illinois, 60613, United States',
 'class': 'leisure',
 'type': 'stadium',
 'importance': 0.7077770838054844}